<a href="https://colab.research.google.com/github/UoR-Vision/VisualDota/blob/main/src/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment Setting

***
<a id="colab"></a>
## Adapting to the Colab Operating Environment

In Colab environment, it is neccessary to download the whole project from Github.

In [1]:
def is_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

if is_colab():
    print("Running in Google Colab!")
    !git clone https://github.com/UoR-Vision/VisualDota.git -b main
    %cd VisualDota/src
else:
    print("Not running in Colab.")

Running in Google Colab!
Cloning into 'VisualDota'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 47 (delta 2), reused 0 (delta 0), pack-reused 38 (from 2)
Receiving objects: 100% (47/47), 114.31 MiB | 14.87 MiB/s, done.
Resolving deltas: 100% (2/2), done.
Updating files: 100% (29/29), done.
/content/VisualDota/src


***
<a id="hotload"></a>
## Hot Loading .py File

Here is to import custom .py file. It can be hot-loaded, just re-run the following cell.

Documentation comments are provided for each public function. To check the comments please use the help() function.

In [ ]:
!pip install importlib
import importlib  # for hot-loaded .py file

try:
    import data_process  # import custom methods for data pre-processing
    import visual_utils  # import custom methods for visualization
except ModuleNotFoundError as e:
    print(f'[ERROR] fail to import a custom module at path: {os.path.abspath(".")}, msg: {e}.')
except Exception as e:
    print(f'[ERROR] fail to read module at path: {os.path.abspath(".")}, msg: {e}')
else:
    # Hot loading python files
    importlib.reload(data_process)
    importlib.reload(visual_utils)

# Data Selection, Exploratory Data Analysis & Preprocessing

***
<a id="import"></a>
## Import libraries

To import the required libraries for whole project.

In [ ]:
# import the build-in required libraries
import os
import warnings

# import the third-part required libraries
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

# check the versions of third-part libraries
print(f'matplotlib: {matplotlib.__version__}, numpy: {np.__version__}')

# 2 Data Selection & Preprocessing

## Load Dataset

In [ ]:
dataset = data_process.load_dataset()
len(dataset)

In [ ]:
visual_utils.visualize_images(dataset)

In [ ]:
labels = data_process.count_labels()
labels

# 3 Exploratory data analysis

# Modelling

# Evaluation

# Summary